# PySMO Surrogate save/load example

## Imports and logging

In [1]:
import logging
logging.getLogger("idaes").setLevel(logging.ERROR)

In [2]:
import pysmo_surrogate as ps
import json
from pandas import DataFrame
from sys import stdout
from io import StringIO
import pandas as pd

## Input data and parameters

In [3]:
training_data = DataFrame({'x1': [1, 2, 3, 4, 5], 'x2': [5, 6, 7, 8, 9], 'z1': [10, 20, 30, 40, 50], 'z2': [6, 8, 10, 12, 14]})
validation_data = DataFrame({'x1': [1, 2, 3, 4], 'x2': [5, 6, 7, 8], 'z1': [10, 20, 30, 40], 'z2': [6, 8, 10, 12]})
input_labels = ["x1", "x2"]
output_labels = ["z1", "z2"]
bounds = {"x1": (0, 5), "x2": (0, 10)}

## Create surrogate trainer

In [4]:
tr = ps.PysmoPolyTrainer(
    input_labels=input_labels,
    output_labels=output_labels,
    input_bounds=bounds,
    training_dataframe=training_data,
    validation_dataframe=validation_data,
    maximum_polynomial_order = 1, 
    multinomials=False,
    number_of_crossvalidations=5,
    solution_method='pyomo')

# train the surrogate, which returns a TrainedSurrogate object
ts = tr.train_surrogate()

2022-02-25 15:29:14 [WARNING] idaes.surrogate.pysmo.polynomial_regression: Warning: pysmo_poly_z1.pickle already exists; results will be saved to pysmo_poly_z1_v_02-25-22_152914.pickle
2022-02-25 15:29:14 [INFO] idaes.surrogate.pysmo.polynomial_regression: No iterations will be run.
2022-02-25 15:29:14 [INFO] idaes.surrogate.pysmo.polynomial_regression: Parameter estimation method: pyomo
2022-02-25 15:29:14 [INFO] idaes.surrogate.pysmo.polynomial_regression: max_fraction_training_samples set at 0.5
2022-02-25 15:29:14 [INFO] idaes.surrogate.pysmo.polynomial_regression: Number of adaptive samples (no_adaptive_samples) set at 4
2022-02-25 15:29:14 [INFO] idaes.surrogate.pysmo.polynomial_regression: Maximum number of iterations (Max_iter) set at: 0
2022-02-25 15:29:14 [INFO] idaes.surrogate.pysmo.polynomial_regression: Initial surrogate model is of order 1 with a cross-val error of 0.000000
2022-02-25 15:29:14 [INFO] idaes.surrogate.pysmo.polynomial_regression: Initial Regression Model Pe

## Create a surrogate from the TrainedSurrogate object and input data

In [5]:
s = ps.PysmoSurrogate(ts, input_labels, output_labels, input_bounds=bounds)

## Save the trained surrogate (to an in-memory IO stream)

In [6]:
strm = StringIO()
s.save(strm)

## Reload the trained surrogate

In [7]:
strm.seek(0)
s2 = ps.PysmoSurrogate.load(strm)

2022-02-25 15:29:19 [INFO] idaes.pysmo_surrogate: Decode surrogate. type=poly
2022-02-25 15:29:19 [INFO] idaes.surrogate.pysmo.polynomial_regression: The number of cross-validation cases (3) is used.
2022-02-25 15:29:19 [INFO] idaes.surrogate.pysmo.polynomial_regression: The default training/cross-validation split of 0.75 is used.
2022-02-25 15:29:19 [INFO] idaes.surrogate.pysmo.polynomial_regression: No iterations will be run.
2022-02-25 15:29:19 [INFO] idaes.surrogate.pysmo.polynomial_regression: Default parameter estimation method is used.
2022-02-25 15:29:19 [INFO] idaes.surrogate.pysmo.polynomial_regression: Parameter estimation method: pyomo
2022-02-25 15:29:19 [INFO] idaes.surrogate.pysmo.polynomial_regression: The number of cross-validation cases (3) is used.
2022-02-25 15:29:19 [INFO] idaes.surrogate.pysmo.polynomial_regression: The default training/cross-validation split of 0.75 is used.
2022-02-25 15:29:19 [INFO] idaes.surrogate.pysmo.polynomial_regression: No iterations wil

In [8]:
s.evaluate_surrogate(validation_data[['x1', 'x2']])

,z1,z2
0,10.0,6.0
1,20.0,8.0
2,30.0,10.0
3,40.0,12.0


In [9]:
s2.evaluate_surrogate(validation_data[['x1', 'x2']])

,z1,z2
0,10.0,6.0
1,20.0,8.0
2,30.0,10.0
3,40.0,12.0
